In [700]:
from collections import defaultdict
from tqdm import tqdm
import math
import numpy as np

In [701]:
with open("data/train") as f:
    data = f.readlines()

In [702]:
specialChars = {
    # "," : ",",
    # "." : ".", 
    # "$" : "$",
    # "the" : "DT",
    # "``" : "``",
    # "''" : "''",
    # "The" : "DT",
    # "--" : ":", 
    # "%" : "NN"
} 


In [703]:
frequencyWords = defaultdict(int)
frequencyPOS = defaultdict(int)
for d in data:
    s = d.split("\t")
    if len(s) < 2:
        continue

    frequencyWords[s[1]] += 1
    frequencyPOS[s[2][:-1]] += 1
    
frequencyWords = dict(sorted(frequencyWords.items(), key=lambda item: item[1], reverse=True))

In [704]:
# check for words with frequency less than K

k = 1

Vocab = [] # list(frequencyWords.keys())
unkCount = 0
for word, freq in frequencyWords.items():
    if freq > k:
        Vocab.append(word)
    else:
        unkCount += freq
        
print(len(Vocab), unkCount)


23182 20011


In [705]:
rank = 1

with open("vocab.txt", "w+") as f:
    f.write(f"<unk>\t{0}\t{unkCount}\n")
    for key, val in frequencyWords.items():
        if val > k:
            f.write(f"{key}\t{rank}\t{val}\n")
            rank += 1

In [706]:
frequencyPOSNormalized = {}
total = sum(frequencyPOS.values())
for k, v in frequencyPOS.items():
    frequencyPOSNormalized[k] = np.float64(v/total)

POSlist = list(frequencyPOSNormalized.keys())

In [707]:
len(frequencyPOSNormalized)

45

# Make Dict

In [710]:
sentences = []
pos = []

sent = []
postemp = []
for d in tqdm(data):
    s = d.split("\t")
    if len(s) < 2:
        sentences.append(sent)
        pos.append(postemp)
        sent = []
        postemp = []    
        continue
    
    if s[1] not in Vocab:
        sent.append("<unk>")  
    else:
        sent.append(s[1])
    postemp.append(s[2][:-1])
    

100%|██████████| 950312/950312 [01:39<00:00, 9557.36it/s] 


In [711]:
sum(frequencyPOS.values())

912095

In [712]:
emissionCount = defaultdict(int)
transitionCount = defaultdict(int)
stateFreq = defaultdict(int)

for s, p in tqdm(zip(sentences, pos)):
    if len(s) != len(p):
        print("error")
    for i in range(len(p)-1):
        transitionCount[(p[i], p[i+1])] += 1
        emissionCount[(p[i], s[i])] += 1
        stateFreq[p[i]] += 1
    emissionCount[(p[-1], s[-1])] += 1
    stateFreq[p[-1]] += 1

38217it [00:01, 23833.92it/s]


In [714]:
emissionDict = {}
transitionDict = {}

for k, v in tqdm(transitionCount.items()):
    transitionDict[f"{k[0]}|{k[1]}"] = np.float64(int(v) / int(stateFreq[k[0]]))
    
for k, v in tqdm(emissionCount.items()):
    emissionDict[f"{k[0]}|{k[1]}"] = np.float64(int(v) / int(stateFreq[k[0]]))

100%|██████████| 30303/30303 [00:00<00:00, 537429.78it/s]


In [715]:
import json
compiledDict = {"transition" : transitionDict, "emission" : emissionDict}
with open("hmm.json", "w") as outfile:
    json.dump(compiledDict, outfile, indent = 4)

In [716]:
len(transitionDict), len(emissionDict)

(1351, 30303)